# Notes on lblnew.f
In this version of the line-by-line code (or CRD model), the spectral points in the user-specified range are sorted into a number of g-groups, then the flux for each g-group is calculated in two different ways:
1. the normal line-by-line method
2. k-distribution method

In the line-by-line method, effectively, flux is calculated at each spectral point and then added up for each g-group.  On the other hand, in the k-distribution method, absorption coefficient is averaged for each g-group; the flux is then calculated for each g-group using this mean absorption coefficient.  

Both methods yield their own result: flux in each g-group.  The motivation is to compare the results from these two methods, with the line-by-line result taken as the benchmark.

This is essentially a dissection of 'lblnew.f', and aims to document what is being calculated in detail.

Sections:

* <a href="#Membership_function">Membership function</a>  
* <a href="#iband_$\Delta\nu$">iband $\Delta\nu$</a>  
* <a href="#Blackbody_radiation">Blackbody radiation</a>   
* <a href="#Line-by-line_Method"> Line-by-line Method</a>  
  * <a href="#Line-by-line_Method__Transmittance">Transmittance</a>
  * <a href="#Line-by-line_Method__Flux">Flux</a>
* <a href="#k-distribution_Method">k-distribution Method</a>
  * <a href="#k-distribution_Method__k-distribution_function">k-distribution function</a>
  * <a href="#k-distribution_Method__Mean_absorption_coefficient">Mean absorption coefficient</a>
  * <a href="#k-distribution_Method__Transmittance">Transmittance</a>
  * <a href="#k-distribution_Method__Planck's_radiation_intensity">Planck's radiation intensity</a>
  * <a href="#k-distribution_Method__Flux">Flux</a>

<a id="Membership_function"></a>

## Membership function

Define the *membership*, $w$, to be:

\begin{equation}
w[g, \nu] = 
\left \{
\begin{array}{rl}
1 & \text{if } \nu \text{ in g-group } g \\
0 & \text{if } \nu \text{ not in g-group } g
\end{array}
\right . \quad .
\end{equation}

$\nu$ is the wavenumber of a spectral point.  So, $w[g, \nu]$ informs whether wavenumber $\nu$ belongs to g-group $g$.

Since each wavenumber can only belong to one g-group:

\begin{equation}
\sum_{g} w[g, \nu] = 1 \quad .
\end{equation}


The total number of spectral points (or wavenumbers) is

\begin{equation}
\sum_{\nu} \sum_{g} w[g, \nu] = \sum_{\nu} 1 \quad .
\end{equation}

<a id="iband_$\Delta\nu$"></a>

## iband $\Delta\nu$

In the code the integration over the whole input spectral range is split up into small intervals, most often with a width of $1 \: cm^{-1}$, each indexed by `iband`.  Let $\Delta\nu$ denote `iband` in the code.

This implies that in the code `gk` is

\begin{equation}
\mathrm{gk}[g, \Delta\nu] = \sum_{\nu}^{\Delta\nu} w[g, \nu]
\end{equation}

It is the number of wavenumbers that belong to g-group $g$ in the iband $\Delta\nu$.

<a id="Blackbody_radiation"></a>

## Blackbody radiation

The spectral intensity (spectral radiance integrated over wavenumbers) over iband $\Delta\nu$ is 

\begin{equation}
B[T, \Delta\nu] = 3.74\times 10^{-5} (\nu_{\Delta\nu})^3 \cdot 0.001 \Delta_{\Delta\nu} \bigg/ \biggl( \exp{\frac{1.4385 (\nu_{\Delta\nu})}{T} - 1} \biggr) \quad ,
\end{equation}
where $\nu_{\Delta\nu}$ is the wavenumber at the centre of the iband $\Delta\nu$, and $\Delta_{\Delta\nu}$ is the width of the iband.

In the code, this is:

```
rsub(iband) = 3.74e-5_r8 *
     &        vm * vm * vm / (exp(1.4385_r8 * vm / 250.0_r8) - 1._r8)
     &        * vd * 0.001_r8
```

`rtotalband` is

\begin{equation}
\sum_{\Delta\nu} B[250K, \Delta\nu] \bigg/ \sum_{\Delta\nu} 1 \quad .
\end{equation}

 <a id="Line-by-line_Method"></a>

## Line-by-line Method

<a id="Line-by-line_Method__Transmittance"></a>

### Transmittance 

By taking narrow enough ibands, the calculation is akin enough to a line-by-line calculation, even if we average the transmittance over the iband first.  

The optical depth at wavenumber $\nu$ and layer $l$ is

\begin{equation}
\tau[\nu, l] = k[\nu, l] \: \omega[l] \quad,
\end{equation}
where $\omega[l]$ is the absorber amount of layer $l$.

The total optical depth between layers $l_{1}$ and $l_{2}$ is 

\begin{equation}
\sum_{l'=l_1}^{l_2} \tau[\nu, l']
\end{equation}

The transmittance associated with this optical depth (or `tku` returned by `transf()` in the code) is  

\begin{equation}
e^{-\sum_{l'=l_1}^{l_2} \tau[\nu, l']}
\end{equation}

Now we average this over the iband $\Delta\nu$ for each g-group, giving (`trang` in the code):

\begin{equation}
\overline{T}[g, l_1, l_2, \Delta\nu] =  \sum_{\nu}^{\Delta\nu} w[g, \nu] e^{-\sum_{l'=l_1}^{l_2} \tau[\nu, l']} 
\bigg/ \sum_{\nu}^{\Delta\nu} w[g, \nu] \quad .
\end{equation}

Notice that the averaging is that of transmittances, rather than absorption coeficients.  And again, even though this is not at a single wavenumber $\nu$, the small enough $\Delta\nu$ makes it close to a line-by-line claculation.






<a id="Line-by-line_Method__Flux"></a>

### Flux

For fluxes, the index $l$ denotes atmosphere level; $np$ is the number of atmosphere layers.

We use the iband-averaged transmittance to calculate the fluxes as follows:

\begin{equation}
F_{\uparrow}[g, l, \Delta\nu] = \left\{
\begin{array}{lr}
\sum_{l'> l} \bigl[ \overline{T}[g, l, l', \Delta\nu] \:
\bigl( B_{\uparrow}[l'-1, \Delta\nu] - B_{\uparrow}[l', \Delta\nu] \bigr) \bigr] - B_{\uparrow}[l, \Delta\nu] 
& \text{if } l = 1, 2, \dots, np \\
- B[l, \Delta\nu] &  \text{if } l = np + 1
\end{array}
\right. \quad .
\end{equation}

\begin{equation}
F_{\downarrow}[g, l, \Delta\nu] = \left\{
\begin{array}{lr}
\sum_{l'< l} \bigl[ \overline{T}[g, l, l', \Delta\nu] \:
\bigl( B_{\downarrow}[l'-1, \Delta\nu] - B_{\downarrow}[l', \Delta\nu] \bigr) \bigr] + B_{\downarrow}[l-1, \Delta\nu] 
& \text{if } l = 2, 3, \dots, np + 1 \\
0 & \text{if } l = 1
\end{array}
\right. \quad .
\end{equation}

$F_{\uparrow}[g, l, \Delta\nu]$ and $F_{\downarrow}[g, l, \Delta\nu]$ are `flu` and `fld` returned by `irflux()`, and 
$B_{\uparrow}[l, \Delta\nu]$ and $B_{\downarrow}[l, \Delta\nu]$ are `bu` and `bd` in `irflux()`; these are Planck's spectral intensities interpolated for upper and lower parts of atmosphere layers.

These iband fluxes are then combined using a weighted sum as follows to give the total fluxes for each g-group:

\begin{equation}
F_{\uparrow\downarrow}[g, l] = \sum_{\Delta\nu} F_{\uparrow\downarrow}[g, l, \Delta\nu] 
\biggl( \sum_{\nu}^{\Delta\nu} w[g, \nu] \bigg/ \sum_{\nu}^{\Delta\nu} 1 \biggr) \quad .
\end{equation}

These are `flug` and `fldg` in the code, and are taken to be the fluxes in each g-group calculated using the *line-by-line* method, the most accurate.


<a id="k-distribution_Method"></a>

## k-distribution Method

<a id="k-distribution_Method__k-distribution_function"></a>

### k-distribution function

Let $\kappa$ denote the k-distribution function.  It is the proportion of spectral points belonging to g-group $g$:

\begin{equation}
\kappa[g] = \sum_{\nu} w[g,\nu] \bigg/ \sum_{\nu} 1 \quad .
\end{equation}


The Planck-weighted k-distribution function is:

\begin{equation}
\kappa_{planck}[g] = \sum_{\Delta\nu} 
\frac{\sum_{\nu}^{\Delta\nu} w[g,\nu] \: B[250 K, \Delta\nu]}
{\sum_{\Delta\nu} \: B[250K, \Delta\nu]} 
\frac{\sum_{\Delta\nu} 1}{\sum_{\nu} 1} \quad .
\end{equation}


In the code, $\kappa$ is denoted $\mathrm{dg}$ while $\kappa_{planck}$ is denoted $\mathrm{dgs}$.  Note that $(\sum_{\Delta\nu} 1)$ is the number of ibands.

<a id="k-distribution_Method__Mean_absorption_coefficient"></a>

### Mean absorption coefficient
The absorption coefficient is a function of wavenumber, pressure and temperature.  Since each layer of the atmosphere has its pressure and temperature, we often denote the absorption coefficient by $k[\nu, l]$.  By taking a linear average, the mean absorption coefficient over g-group $g$ is

\begin{equation}
\overline{k}[g, l] = \sum_{\nu} w[g, k] k[\nu, l] \bigg/ \sum_{\nu} w[g, \nu]    \quad .
\end{equation}

This is equivalent to what is computed in the code:

\begin{equation} 
\mathrm{absmean}[g, l] = \frac{\sum_{\Delta\nu} \mathrm{gk}[g, \Delta\nu] 
\: k[\nu, l]}
{\mathrm{dg}[g] \sum_{\nu} 1} \quad .
\end{equation}

<a id="k-distribution_Method__Transmittance"></a>

### Transmittance
Starting from the mean $k$ over all spectral points, $\overline{k}[g, l]$, we can proceed in a similar fashion to calculate the fluxes, the main difference being that there is no wavenumber-dependence from this point onwards (from the mean $k$) in the calculation.

The transmittance is now
\begin{equation}
\overline{T}[g, l_1, l_2] = 
e^{-\sum_{l'=l_1}^{l_2} \overline{k}[g, l'] \: \omega[l']}  \quad .
\end{equation}

This is `tku`, or `tran[l1,l2]`, in the code.

<a id="k-distribution_Method__Planck's_radiation_intensity"></a>

### Planck's radiation intensity

In the k-distribution method, the Planck's radiation intensity for the entire range spanned all the spectral points is used:

\begin{equation}
Z[l]  = \sum_{\Delta\nu} B[l, \Delta\nu] \quad .
\end{equation}

<a id="k-distribution_Method__Flux"></a>

### Flux
The form of expression for the fluxes is same as before, except now the terms are for the entire spectral range and not for a small spectral range iband.

\begin{equation}
F_{\uparrow}[g, l] = \left\{
\begin{array}{lr}
\sum_{l'> l} \bigl[ \overline{T}[g, l, l'] \:
\bigl( B_{\uparrow}[l'-1] - B_{\uparrow}[l'] \bigr) \bigr] - B_{\uparrow}[l] 
& \text{if } l = 1, 2, \dots, np \\
- B[l] &  \text{if } l = np + 1
\end{array}
\right. \quad .
\end{equation}

\begin{equation}
F_{\downarrow}[g, l] = \left\{
\begin{array}{lr}
\sum_{l'< l} \bigl[ \overline{T}[g, l, l'] \:
\bigl( B_{\downarrow}[l'-1] - B_{\downarrow}[l'] \bigr) \bigr] + B_{\downarrow}[l-1] 
& \text{if } l = 2, 3, \dots, np + 1 \\
0 & \text{if } l = 1
\end{array}
\right. \quad .
\end{equation}

$B_{\uparrow}[l]$ and $B_{\downarrow}[l]$ are `bu` and `bd` in `irflux()`; these are Planck's spectral intensities, $Z[l]$, interpolated for upper and lower parts of atmosphere layers

The final fluxes are scaled by the Planck-weighted k-distribution function

\begin{equation}
F_{\uparrow\downarrow}[g, l] = F_{\uparrow\downarrow}[g, l] \cdot \kappa_{planck}[g]
\end{equation}